In [9]:
import os 
%pwd

'/home/prathameshdevadiga/personal/projs/KidneyDiseaseClassification/research'

In [10]:
os.chdir("../")

In [11]:
%pwd

'/home/prathameshdevadiga/personal/projs/KidneyDiseaseClassification'

In [12]:
os.environ["MLFLOW_TRACKING_URI"] = "https://dagshub.com/devadigapratham8/KidneyDiseaseClassification.mlflow"
os.environ['MLFLOW_TRACKING_USERNAME'] = "devadigapratham8" 
os.environ["MLFLOW_TRACKING_PASSWORD"] = "328fa8e4b55771d3b3a06a004515e457f75865d9"

In [13]:
import tensorflow as tf 

In [14]:
model = tf.keras.models.load_model("artifacts/training/model.h5")

2024-01-02 20:39:23.677464: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-01-02 20:39:23.810446: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [15]:
from dataclasses import dataclass 
from pathlib import Path 

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict 
    mlflow_uri: str 
    params_image_size: list 
    params_batch_size: int 


In [16]:
from cnnClassifier.constants import * 
from cnnClassifier.utils.common import read_yaml, create_directories, save_json

In [21]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH, 
            params_filepath = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath) 
        self.params = read_yaml(params_filepath) 

        create_directories([self.config.artifacts_root])

    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model = "artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone",
            mlflow_uri="https://dagshub.com/devadigapratham8/KidneyDiseaseClassification.mlflow", 
            all_params=self.params, 
            params_image_size=self.params.IMAGE_SIZE, 
            params_batch_size=self.params.BATCH_SIZE
        )

        return eval_config 

In [22]:
import tensorflow as tf 
from pathlib import Path 
import mlflow 
import mlflow.keras 
from urllib.parse import urlparse 

In [23]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    
    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )


    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    
    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )
            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
            else:
                mlflow.keras.log_model(self.model, "model")


In [24]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
   raise e

[2024-01-02 20:40:21,919 : INFO: common: yaml file : config/config.yaml loaded successfully]
[2024-01-02 20:40:21,926 : INFO: common: yaml file : params.yaml loaded successfully]
[2024-01-02 20:40:21,928 : INFO: common: Created directory at : artifacts]
Found 3733 images belonging to 1 classes.


2024-01-02 20:40:23.788343: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
2024-01-02 20:40:24.534729: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 205520896 exceeds 10% of free system memory.
2024-01-02 20:40:24.717561: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 205520896 exceeds 10% of free system memory.
2024-01-02 20:40:25.464732: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 51380224 exceeds 10% of free system memory.
2024-01-02 20:40:25.555224: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 102760448 exceeds 10% of free system memory.
2024-01-02 20:40:25.869412: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 102760448 exceeds 10

234/234 [==============================] - 1512s 6s/step - loss: 118.4305 - accuracy: 1.0000
[2024-01-02 21:05:36,011 : INFO: common: Json file saved at scores.json]


2024/01/02 21:05:38 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


[2024-01-02 21:05:40,432 : WARNING: save: Found untraced functions such as _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op while saving (showing 5 of 14). These functions will not be directly callable after loading.]
[2024-01-02 21:05:41,190 : INFO: builder_impl: Assets written to: /tmp/tmp41t72_k_/model/data/model/assets]


Successfully registered model 'VGG16Model'.
2024/01/02 21:06:13 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: VGG16Model, version 1
Created version '1' of model 'VGG16Model'.
